In [2]:
from sklearn.externals import joblib
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [13]:
from sagemaker import get_execution_role
import boto3
from s3fs.core import S3FileSystem
s3 = S3FileSystem(anon=False)
role = get_execution_role()
bucket='instacartresearchsymposium'
mapper = joblib.load("userProductTotal_pivot_sparce_mapper")
userProductTotal_pivot_sparce = joblib.load("userProductTotal_pivot_sparce")
data_key = 'orig_products.pickle'
products = joblib.load(s3.open('{}/{}'.format(bucket, data_key), mode='rb'))

In [8]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(userProductTotal_pivot_sparce)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=20, p=2, radius=1.0)

In [5]:
# create mapper from movie title to index


def getIndexForProductID(productID): 
    return mapper.loc[mapper.ProductID == productID]["index"].values[0]
getIndexForProductID(23)

4

In [9]:
def makeProductRecommendation(model, data, productID, numRecommendations):
    model.fit(data)
    print("Product ID inserted:", productID, products[products.ProductID == productID]["ProductName"].to_string(index=False))
    print("Recommending....")
    distances,indices = model.kneighbors(data[getIndexForProductID(productID)], n_neighbors = numRecommendations + 1)
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # print recommendations
    #print(raw_recommends)
    for i, (productID, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, products[products.ProductID == productID]["ProductName"].to_string(index=False), dist))

In [14]:
makeProductRecommendation(knn,userProductTotal_pivot_sparce, 21137, 5)

Product ID inserted: 21137  Organic Strawberries
Recommending....
1:  Unsweetened Strawberry Kiwi Water, with distance of 0.5410525998524192
2:  Garlic Asiago Brazilian Cheese Bread, with distance of 0.5248754978423431
3:  Chicken & vegetable dumplings with rice in an ..., with distance of 0.5009665165843328
4:  Original English Muffins, with distance of 0.4858102812322225
5:  Mung Bean Pasta, with distance of 0.4801123950004902


In [15]:
makeProductRecommendation(knn,userProductTotal_pivot_sparce, 24852, 5)

Product ID inserted: 24852  Banana
Recommending....
1:  Unsweetened Strawberry Kiwi Water, with distance of 0.6378966888208069
2:  Coffee, Gourmet, All Purpose Grind, Medium, Ha..., with distance of 0.6167822731300237
3:  Madagascar Vanilla Ice Cream, with distance of 0.6076141034735598
4:  Total 2% with Raspberry Pomegranate Lowfat Gre..., with distance of 0.6042202321080975
5:  Raisin Brown Bread, with distance of 0.5749582172432766


In [16]:
makeProductRecommendation(knn,userProductTotal_pivot_sparce, 21903, 5)

Product ID inserted: 21903  Organic Baby Spinach
Recommending....
1:  Original English Muffins, with distance of 0.5912642455707927
2:  Organic Diced with Garlic & Onion Tomatoes, with distance of 0.5847140747533568
3:  Mung Bean Pasta, with distance of 0.5763691622348515
4:  Simply Clean Fragrance Free Wipes, with distance of 0.563443127112754
5:  Coffee, Gourmet, All Purpose Grind, Medium, Ha..., with distance of 0.5410525998524192


In [17]:
makeProductRecommendation(knn,userProductTotal_pivot_sparce, 47766, 5)

Product ID inserted: 47766  Organic Avocado
Recommending....
1:  Simply Clean Fragrance Free Wipes, with distance of 0.6746344060337235
2:  Organic Popped Corn Simply Salted, with distance of 0.6441866568203056
3:  Unsweetened Strawberry Kiwi Water, with distance of 0.6193569531747778
4:  Organic Daikon Radish Bunch, with distance of 0.6076141034735598
5:  Total 2% with Raspberry Pomegranate Lowfat Gre..., with distance of 0.5754492045508461
